# Data API to AWS RDS: retrieve, clean, upload
## Step 1:
- Create an HTTP request to the 2021 Yellow Taxi Trip Data API
- Parse the JSON response to extract the information
- Set query parameters for pagination and field selection
- Clean retrieved data: 
    - Convert the timestamp string to a datetime object
    - Extract the date and time components and create new columns
    - Delete unnecessary columns
    - Delete rows with null values
    - Create a column with unique keys
    - Rearrange columns
    - Save clean data to a CSV file.


#### Import necessary libraries

In [93]:
import requests
import json
import pandas as pd

#### Add the URL we want to make the request to

In [94]:
url = 'https://data.cityofnewyork.us/resource/m6nq-qud6.json'

#### Set the limit to retrieve the first record

In [95]:
limit = 1
offset = 0
params = {
    '$limit': limit,
    '$offset': offset
}

 #### Receive the status code for our request

In [96]:
response = requests.get(url, params=params)
response.status_code  

200

#### Read the data we received back from the API

In [97]:
response.json()

[{'vendorid': '1',
  'tpep_pickup_datetime': '2021-01-01T00:30:10.000',
  'tpep_dropoff_datetime': '2021-01-01T00:36:12.000',
  'passenger_count': '1',
  'trip_distance': '2.10',
  'ratecodeid': '1',
  'store_and_fwd_flag': 'N',
  'pulocationid': '142',
  'dolocationid': '43',
  'payment_type': '2',
  'fare_amount': '8',
  'extra': '3',
  'mta_tax': '0.5',
  'tip_amount': '0',
  'tolls_amount': '0',
  'improvement_surcharge': '0.3',
  'total_amount': '11.8',
  'congestion_surcharge': '2.5'}]

#### Set query parameters for pagination and field selection. At the end, print it out the DataFrame

In [99]:
limit = 100000  # Number of records per page. Adjust the limit as needed
offset = 0
data = []

while True:    
    params = {
        '$limit': limit,
        '$offset': offset,
        '$select': 'tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,payment_type,total_amount',
        '$where': 'total_amount >= 50'
    }    
   
    response = requests.get(url, params=params)    
    
    if response.status_code == 200:
        
        # Load the data as a JSON object
        batch_data = response.json()

        # If batch_data is empty, break the loop
        if not batch_data:
            print('All data retrieved')
            break

        # Extend the data list with the batch_data
        data.extend(batch_data)

        # Update the offset for the next batch
        offset += limit

    else:
        print(f"Request failed with status code {response.status_code}")
        break

# Create a pandas DataFrame from the JSON data
df = pd.DataFrame(data)
print('All data retrieved')
df

All data retrieved


,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,payment_type,total_amount
0,2021-01-01T01:48:35.000,2021-01-01T02:26:28.000,1,0.00,1,50
1,2021-01-01T19:26:18.000,2021-01-01T19:50:57.000,4,15.80,1,50
2,2021-01-01T21:18:36.000,2021-01-01T21:42:55.000,2,16.09,1,50
3,2021-01-02T11:25:11.000,2021-01-02T11:43:54.000,1,11.16,1,50
4,2021-01-02T15:52:17.000,2021-01-02T16:19:35.000,1,10.50,1,50
...,...,...,...,...,...,...
1673894,2021-02-04T17:25:03.000,2021-02-04T17:47:01.000,1,2.62,2,6969.3
1673895,2021-01-04T16:04:51.000,2021-01-04T16:15:01.000,1,2.05,1,7661.28
1673896,2021-04-10T13:14:49.000,2021-04-10T13:50:53.000,1,5.70,3,395854.74
1673897,2021-03-18T12:10:41.000,2021-03-18T12:20:03.000,1,0.00,3,398469.2


#### Count the number of rows

In [100]:
len(df)

1673899

#### Convert the timestamp string to a datetime object

In [101]:
df['pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

#### Extract the date and time components and create new columns

In [102]:
df['pickup_date'] = df['pickup_datetime'].dt.date
df['pickup_time'] = df['pickup_datetime'].dt.time

df['dropoff_date'] = df['dropoff_datetime'].dt.date
df['dropoff_time'] = df['dropoff_datetime'].dt.time

df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,payment_type,total_amount,pickup_datetime,dropoff_datetime,pickup_date,pickup_time,dropoff_date,dropoff_time
0,2021-01-01T01:48:35.000,2021-01-01T02:26:28.000,1,0.00,1,50,2021-01-01 01:48:35,2021-01-01 02:26:28,2021-01-01,01:48:35,2021-01-01,02:26:28
1,2021-01-01T19:26:18.000,2021-01-01T19:50:57.000,4,15.80,1,50,2021-01-01 19:26:18,2021-01-01 19:50:57,2021-01-01,19:26:18,2021-01-01,19:50:57
2,2021-01-01T21:18:36.000,2021-01-01T21:42:55.000,2,16.09,1,50,2021-01-01 21:18:36,2021-01-01 21:42:55,2021-01-01,21:18:36,2021-01-01,21:42:55
3,2021-01-02T11:25:11.000,2021-01-02T11:43:54.000,1,11.16,1,50,2021-01-02 11:25:11,2021-01-02 11:43:54,2021-01-02,11:25:11,2021-01-02,11:43:54
4,2021-01-02T15:52:17.000,2021-01-02T16:19:35.000,1,10.50,1,50,2021-01-02 15:52:17,2021-01-02 16:19:35,2021-01-02,15:52:17,2021-01-02,16:19:35
...,...,...,...,...,...,...,...,...,...,...,...,...
1673894,2021-02-04T17:25:03.000,2021-02-04T17:47:01.000,1,2.62,2,6969.3,2021-02-04 17:25:03,2021-02-04 17:47:01,2021-02-04,17:25:03,2021-02-04,17:47:01
1673895,2021-01-04T16:04:51.000,2021-01-04T16:15:01.000,1,2.05,1,7661.28,2021-01-04 16:04:51,2021-01-04 16:15:01,2021-01-04,16:04:51,2021-01-04,16:15:01
1673896,2021-04-10T13:14:49.000,2021-04-10T13:50:53.000,1,5.70,3,395854.74,2021-04-10 13:14:49,2021-04-10 13:50:53,2021-04-10,13:14:49,2021-04-10,13:50:53
1673897,2021-03-18T12:10:41.000,2021-03-18T12:20:03.000,1,0.00,3,398469.2,2021-03-18 12:10:41,2021-03-18 12:20:03,2021-03-18,12:10:41,2021-03-18,12:20:03


#### Delete unnecessary columns

In [103]:
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pickup_datetime', 'dropoff_datetime'], axis=1)
df

,passenger_count,trip_distance,payment_type,total_amount,pickup_date,pickup_time,dropoff_date,dropoff_time
0,1,0.00,1,50,2021-01-01,01:48:35,2021-01-01,02:26:28
1,4,15.80,1,50,2021-01-01,19:26:18,2021-01-01,19:50:57
2,2,16.09,1,50,2021-01-01,21:18:36,2021-01-01,21:42:55
3,1,11.16,1,50,2021-01-02,11:25:11,2021-01-02,11:43:54
4,1,10.50,1,50,2021-01-02,15:52:17,2021-01-02,16:19:35
...,...,...,...,...,...,...,...,...
1673894,1,2.62,2,6969.3,2021-02-04,17:25:03,2021-02-04,17:47:01
1673895,1,2.05,1,7661.28,2021-01-04,16:04:51,2021-01-04,16:15:01
1673896,1,5.70,3,395854.74,2021-04-10,13:14:49,2021-04-10,13:50:53
1673897,1,0.00,3,398469.2,2021-03-18,12:10:41,2021-03-18,12:20:03


#### Add a new column 'trip_id' with unique values

In [104]:
import uuid

df['trip_id'] = [str(uuid.uuid4()) for _ in range(len(df))]
df

,passenger_count,trip_distance,payment_type,total_amount,pickup_date,pickup_time,dropoff_date,dropoff_time,trip_id
0,1,0.00,1,50,2021-01-01,01:48:35,2021-01-01,02:26:28,9cedea19-fc75-47dd-899c-ec0c2a18d232
1,4,15.80,1,50,2021-01-01,19:26:18,2021-01-01,19:50:57,4e704911-38e3-489d-85bb-b5c39f206d5e
2,2,16.09,1,50,2021-01-01,21:18:36,2021-01-01,21:42:55,8636e055-617b-4c0a-8120-b7c2c8f03c08
3,1,11.16,1,50,2021-01-02,11:25:11,2021-01-02,11:43:54,899a3d58-7faf-4a20-a085-10d0484c515f
4,1,10.50,1,50,2021-01-02,15:52:17,2021-01-02,16:19:35,2c31a856-9f52-4d19-bfaf-3572779b0dc2
...,...,...,...,...,...,...,...,...,...
1673894,1,2.62,2,6969.3,2021-02-04,17:25:03,2021-02-04,17:47:01,0a3b48a3-d8e8-4453-bda7-da05754cc809
1673895,1,2.05,1,7661.28,2021-01-04,16:04:51,2021-01-04,16:15:01,3ee6dce3-057c-45d0-96fd-90d575b36423
1673896,1,5.70,3,395854.74,2021-04-10,13:14:49,2021-04-10,13:50:53,ceee55d1-9029-43c8-bc90-7296931d38a4
1673897,1,0.00,3,398469.2,2021-03-18,12:10:41,2021-03-18,12:20:03,3f3af158-ccf3-4286-83f0-0c756e3fda79


#### Find and count rows with null values

In [105]:
num_rows_with_null = df.isnull().any(axis=1).sum()

print(f'Rows with null values: {num_rows_with_null}')

Rows with null values: 221318


#### Delete rows with null values

In [106]:
df.dropna(inplace=True)
num_rows_with_null = df.isnull().any(axis=1).sum()

print(f'Rows with null values: {num_rows_with_null}')

Rows with null values: 0


#### Check datatypes

In [107]:
df.dtypes

passenger_count    object
trip_distance      object
payment_type       object
total_amount       object
pickup_date        object
pickup_time        object
dropoff_date       object
dropoff_time       object
trip_id            object
dtype: object

####  Cast the columns to a numeric data type

In [108]:
df['passenger_count'] = pd.to_numeric(df['passenger_count'], errors='coerce')
df['payment_type'] = pd.to_numeric(df['payment_type'], errors='coerce')

df.dtypes

passenger_count    float64
trip_distance       object
payment_type         int64
total_amount        object
pickup_date         object
pickup_time         object
dropoff_date        object
dropoff_time        object
trip_id             object
dtype: object

#### Cast 'passenger_count' object to an int

In [109]:
df['passenger_count'] = df['passenger_count'].astype(int)

df.dtypes
df

,passenger_count,trip_distance,payment_type,total_amount,pickup_date,pickup_time,dropoff_date,dropoff_time,trip_id
0,1,0.00,1,50,2021-01-01,01:48:35,2021-01-01,02:26:28,9cedea19-fc75-47dd-899c-ec0c2a18d232
1,4,15.80,1,50,2021-01-01,19:26:18,2021-01-01,19:50:57,4e704911-38e3-489d-85bb-b5c39f206d5e
2,2,16.09,1,50,2021-01-01,21:18:36,2021-01-01,21:42:55,8636e055-617b-4c0a-8120-b7c2c8f03c08
3,1,11.16,1,50,2021-01-02,11:25:11,2021-01-02,11:43:54,899a3d58-7faf-4a20-a085-10d0484c515f
4,1,10.50,1,50,2021-01-02,15:52:17,2021-01-02,16:19:35,2c31a856-9f52-4d19-bfaf-3572779b0dc2
...,...,...,...,...,...,...,...,...,...
1673894,1,2.62,2,6969.3,2021-02-04,17:25:03,2021-02-04,17:47:01,0a3b48a3-d8e8-4453-bda7-da05754cc809
1673895,1,2.05,1,7661.28,2021-01-04,16:04:51,2021-01-04,16:15:01,3ee6dce3-057c-45d0-96fd-90d575b36423
1673896,1,5.70,3,395854.74,2021-04-10,13:14:49,2021-04-10,13:50:53,ceee55d1-9029-43c8-bc90-7296931d38a4
1673897,1,0.00,3,398469.2,2021-03-18,12:10:41,2021-03-18,12:20:03,3f3af158-ccf3-4286-83f0-0c756e3fda79


#### Rearrange columns

In [110]:
df = df[['trip_id', 'pickup_date', 'pickup_time', 'dropoff_date', 'dropoff_time', 'passenger_count', 'trip_distance', 'payment_type', 'total_amount']]
df

,trip_id,pickup_date,pickup_time,dropoff_date,dropoff_time,passenger_count,trip_distance,payment_type,total_amount
0,9cedea19-fc75-47dd-899c-ec0c2a18d232,2021-01-01,01:48:35,2021-01-01,02:26:28,1,0.00,1,50
1,4e704911-38e3-489d-85bb-b5c39f206d5e,2021-01-01,19:26:18,2021-01-01,19:50:57,4,15.80,1,50
2,8636e055-617b-4c0a-8120-b7c2c8f03c08,2021-01-01,21:18:36,2021-01-01,21:42:55,2,16.09,1,50
3,899a3d58-7faf-4a20-a085-10d0484c515f,2021-01-02,11:25:11,2021-01-02,11:43:54,1,11.16,1,50
4,2c31a856-9f52-4d19-bfaf-3572779b0dc2,2021-01-02,15:52:17,2021-01-02,16:19:35,1,10.50,1,50
...,...,...,...,...,...,...,...,...,...
1673894,0a3b48a3-d8e8-4453-bda7-da05754cc809,2021-02-04,17:25:03,2021-02-04,17:47:01,1,2.62,2,6969.3
1673895,3ee6dce3-057c-45d0-96fd-90d575b36423,2021-01-04,16:04:51,2021-01-04,16:15:01,1,2.05,1,7661.28
1673896,ceee55d1-9029-43c8-bc90-7296931d38a4,2021-04-10,13:14:49,2021-04-10,13:50:53,1,5.70,3,395854.74
1673897,3f3af158-ccf3-4286-83f0-0c756e3fda79,2021-03-18,12:10:41,2021-03-18,12:20:03,1,0.00,3,398469.2


#### Convert 'trip_distance' from miles to kilomiters

In [111]:
df['trip_distance'] = pd.to_numeric(df['trip_distance'], errors='coerce')
df['trip_distance'] = df['trip_distance'] * 1.60934
df = df.rename(columns={'trip_distance': 'trip_distance_km'})

df

,trip_id,pickup_date,pickup_time,dropoff_date,dropoff_time,passenger_count,trip_distance_km,payment_type,total_amount
0,9cedea19-fc75-47dd-899c-ec0c2a18d232,2021-01-01,01:48:35,2021-01-01,02:26:28,1,0.000000,1,50
1,4e704911-38e3-489d-85bb-b5c39f206d5e,2021-01-01,19:26:18,2021-01-01,19:50:57,4,25.427572,1,50
2,8636e055-617b-4c0a-8120-b7c2c8f03c08,2021-01-01,21:18:36,2021-01-01,21:42:55,2,25.894281,1,50
3,899a3d58-7faf-4a20-a085-10d0484c515f,2021-01-02,11:25:11,2021-01-02,11:43:54,1,17.960234,1,50
4,2c31a856-9f52-4d19-bfaf-3572779b0dc2,2021-01-02,15:52:17,2021-01-02,16:19:35,1,16.898070,1,50
...,...,...,...,...,...,...,...,...,...
1673894,0a3b48a3-d8e8-4453-bda7-da05754cc809,2021-02-04,17:25:03,2021-02-04,17:47:01,1,4.216471,2,6969.3
1673895,3ee6dce3-057c-45d0-96fd-90d575b36423,2021-01-04,16:04:51,2021-01-04,16:15:01,1,3.299147,1,7661.28
1673896,ceee55d1-9029-43c8-bc90-7296931d38a4,2021-04-10,13:14:49,2021-04-10,13:50:53,1,9.173238,3,395854.74
1673897,3f3af158-ccf3-4286-83f0-0c756e3fda79,2021-03-18,12:10:41,2021-03-18,12:20:03,1,0.000000,3,398469.2


#### Rount 'trip_distance' to 2 decimal places

In [112]:
df.dtypes

trip_id              object
pickup_date          object
pickup_time          object
dropoff_date         object
dropoff_time         object
passenger_count       int32
trip_distance_km    float64
payment_type          int64
total_amount         object
dtype: object

In [113]:
df['trip_distance_km'] = df['trip_distance_km'].round(2)
df

,trip_id,pickup_date,pickup_time,dropoff_date,dropoff_time,passenger_count,trip_distance_km,payment_type,total_amount
0,9cedea19-fc75-47dd-899c-ec0c2a18d232,2021-01-01,01:48:35,2021-01-01,02:26:28,1,0.00,1,50
1,4e704911-38e3-489d-85bb-b5c39f206d5e,2021-01-01,19:26:18,2021-01-01,19:50:57,4,25.43,1,50
2,8636e055-617b-4c0a-8120-b7c2c8f03c08,2021-01-01,21:18:36,2021-01-01,21:42:55,2,25.89,1,50
3,899a3d58-7faf-4a20-a085-10d0484c515f,2021-01-02,11:25:11,2021-01-02,11:43:54,1,17.96,1,50
4,2c31a856-9f52-4d19-bfaf-3572779b0dc2,2021-01-02,15:52:17,2021-01-02,16:19:35,1,16.90,1,50
...,...,...,...,...,...,...,...,...,...
1673894,0a3b48a3-d8e8-4453-bda7-da05754cc809,2021-02-04,17:25:03,2021-02-04,17:47:01,1,4.22,2,6969.3
1673895,3ee6dce3-057c-45d0-96fd-90d575b36423,2021-01-04,16:04:51,2021-01-04,16:15:01,1,3.30,1,7661.28
1673896,ceee55d1-9029-43c8-bc90-7296931d38a4,2021-04-10,13:14:49,2021-04-10,13:50:53,1,9.17,3,395854.74
1673897,3f3af158-ccf3-4286-83f0-0c756e3fda79,2021-03-18,12:10:41,2021-03-18,12:20:03,1,0.00,3,398469.2


#### Save clean data to the final CSV file

In [114]:
df.to_csv('ny_taxi_dataset.csv', index=False)
print('ny_taxi_dataset.csv is saved')

ny_taxi_dataset.csv is saved
